In [4]:
#Passo1:Importar bibliotecas
import requests
from bs4 import BeautifulSoup
import pandas as pd 
import time
import urllib.parse
import json
from concurrent.futures import ThreadPoolExecutor, as_completed, ProcessPoolExecutor
import random
import re
from collections import defaultdict
import asyncio
import aiohttp
import threading

# --- PASSO 2: Definir a URL e fazer a requisição inicial ---
url = "https://www.farmaponte.com.br/saude/"
url_base = url

# Header para evitar bloqueio
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36',
    'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8',
    'Accept-Language': 'pt-BR,pt;q=0.8,en;q=0.6',
    'Accept-Encoding': 'gzip, deflate, br',
    'Connection': 'keep-alive',
    'Upgrade-Insecure-Requests': '1',
}

# Criar sessão com pool de conexões otimizado
session = requests.Session()
session.headers.update(headers)
adapter = requests.adapters.HTTPAdapter(
    pool_connections=20,  # Mais conexões simultâneas
    pool_maxsize=50,     # Pool maior
    max_retries=3
)
session.mount('http://', adapter)
session.mount('https://', adapter)

# Controle de rate limiting inteligente
last_request_time = 0
request_lock = threading.Lock()
consecutive_errors = 0

# Cache para evitar requisições duplicadas
url_cache = {}
cache_lock = threading.Lock()

# --- Funções auxiliares OTIMIZADAS ---

def achar_nome(div):
    tag_h2 = div.find('h2', class_='title')
    if tag_h2:
        tag_a = tag_h2.find('a')
        if tag_a:
            return tag_a.get_text(strip=True)
    return "Nome não encontrado"

def achar_preco(div):
    """Busca o preço unitário (preço original)"""
    tag_preco = div.find('p', class_="unit-price")
    if tag_preco:
        return tag_preco.get_text().strip()
    return None

def achar_precopix(div):
    """OTIMIZADO: Busca o preço PIX com método mais eficiente"""
    # Método mais direto primeiro
    for class_name in ['seal-pix sale-price sale-price-pix mb-0 money', 'seal-pix']:
        tag_pix = div.find('p', class_=class_name)
        if tag_pix:
            return tag_pix.get_text().strip()
    return None 

def achar_total_paginas(soup_inicial):
    container_template = soup_inicial.find('div', class_='page-template')
    if container_template:
        divs_candidatas = container_template.find_all('div', class_='text-center pt-3')
        for div in divs_candidatas:
            texto = div.get_text(strip=True)
            if "Página" in texto:
                # Regex mais eficiente
                match = re.search(r'Página\s+\d+\s+de\s+(\d+)', texto)
                if match:
                    return int(match.group(1))
    return 1

def achar_link(produto_html, url_base):
    tag_h2 = produto_html.find('h2', class_='title')
    if tag_h2:
        tag_a = tag_h2.find('a')
        if tag_a and 'href' in tag_a.attrs:
            link_relativo = tag_a.get('href')
            return urllib.parse.urljoin(url_base, link_relativo)
    return None

def achar_precodesconto(produto_html):
    """OTIMIZADO: Busca o preço com desconto de forma mais eficiente"""
    # Método mais direto
    tag_venda = produto_html.find('p', class_="sale-price money")
    if tag_venda:
        strong_tag = tag_venda.find('strong')
        return (strong_tag or tag_venda).get_text(strip=True)
    return None

def achar_desconto_percentual(produto_html):
    """OTIMIZADO: Busca percentual de desconto"""
    discount_element = produto_html.find('span', class_='discount')
    if discount_element:
        return discount_element.get_text(strip=True)
    return None

def limpar_json_string(json_string):
    """OTIMIZADO: Limpeza mais eficiente de JSON"""
    if not json_string:
        return json_string
    
    # Operações mais eficientes em uma só passada
    json_string = re.sub(r'[\n\r\t]+', '', json_string)
    json_string = re.sub(r'\s{2,}', ' ', json_string)
    json_string = re.sub(r',,+', ',', json_string)
    json_string = re.sub(r',\s*([}\]])', r'\1', json_string)
    
    return json_string.strip()

def extrair_detalhes_do_json(soup_detalhes):
    """OTIMIZADO: Extração mais rápida do JSON-LD"""
    brand = None
    code = None
    
    # Buscar apenas o primeiro script JSON-LD relevante
    script_tag = soup_detalhes.find('script', type='application/ld+json')
    if not script_tag or not script_tag.string:
        return brand, code
            
    try:
        raw_json = script_tag.string.strip()
        cleaned_json = limpar_json_string(raw_json)
        dados_json = json.loads(cleaned_json)
        
        # Tratamento simplificado
        if isinstance(dados_json, list) and dados_json:
            for item in dados_json:
                if isinstance(item, dict) and ('brand' in item or 'gtin13' in item):
                    dados_json = item
                    break
        
        # Extração direta
        if isinstance(dados_json, dict):
            # Marca
            brand_data = dados_json.get('brand')
            if isinstance(brand_data, dict):
                brand = brand_data.get('name')
            elif isinstance(brand_data, str):
                brand = brand_data
            
            # Código
            code = dados_json.get('gtin13') or dados_json.get('gtin')
            if code:
                code = str(code)
                
    except:
        # Fallback com regex mais simples
        try:
            raw_json = script_tag.string
            brand_match = re.search(r'"brand":\s*(?:{\s*"name":\s*"([^"]+)"|"([^"]+)")', raw_json)
            if brand_match:
                brand = brand_match.group(1) or brand_match.group(2)
            
            gtin_match = re.search(r'"gtin(?:13)?":\s*"([^"]+)"', raw_json)
            if gtin_match:
                code = gtin_match.group(1)
        except:
            pass
    
    return brand, code

def extrair_detalhes_adicionais_da_pagina(soup_detalhes):
    """OTIMIZADO: Extração mais rápida de detalhes"""
    detalhes = {}
    
    # Buscas mais diretas
    elements = {
        'sub_category': soup_detalhes.find(class_='pr-0 mr-0'),
        'product_code': soup_detalhes.find('span', class_='mr-3'),
        'unit_price_detailed': soup_detalhes.find('p', class_='unit-price'),
        'discount_price_detailed': soup_detalhes.select_one('p.sale-price.money'),
        'pix_price_detailed': soup_detalhes.select_one('p.seal-pix.sale-price.sale-price-pix.mb-0.money')
    }
    
    for key, element in elements.items():
        if element:
            if key == 'pix_price_detailed':
                detalhes[key] = element.get_text(strip=True)
                pix_discount = element.get('data-discount')
                if pix_discount:
                    detalhes['pix_discount_percent'] = pix_discount
            else:
                detalhes[key] = element.get_text(strip=True)
        else:
            detalhes[key] = None
    
    return detalhes

def limpar_preco(preco_str):
    """OTIMIZADO: Limpeza mais rápida de preços"""
    if not preco_str: 
        return None
    try:
        # Regex mais eficiente para extrair números
        match = re.search(r'(\d{1,3}(?:\.\d{3})*(?:,\d{2})?)', preco_str.replace("R$", ""))
        if match:
            preco_limpo = match.group(1).replace('.', '').replace(',', '.')
            return float(preco_limpo)
    except:
        pass
    return None

def baixar_url(url, tentativas=2):
    """OTIMIZADO: Download mais inteligente com rate limiting adaptativo"""
    global last_request_time, consecutive_errors
    
    # Cache check
    with cache_lock:
        if url in url_cache:
            return url_cache[url]
    
    for i in range(tentativas):
        try:
            # Rate limiting inteligente
            with request_lock:
                current_time = time.time()
                time_since_last = current_time - last_request_time
                
                # Delay adaptativo baseado em erros
                if consecutive_errors > 0:
                    delay = min(0.5 + (consecutive_errors * 0.2), 2.0)
                else:
                    delay = 0.1  # Delay mínimo quando tudo está funcionando
                
                if time_since_last < delay:
                    time.sleep(delay - time_since_last)
                
                last_request_time = time.time()
            
            response = session.get(url, timeout=15)
            
            if response.status_code == 200:
                consecutive_errors = 0
                # Cache response
                with cache_lock:
                    url_cache[url] = response
                return response
            elif response.status_code == 429:
                consecutive_errors += 1
                time.sleep(min(2 ** i, 10))  # Exponential backoff
            else:
                consecutive_errors += 1
                
        except Exception as e:
            consecutive_errors += 1
            if i == tentativas - 1:
                print(f"⚠️ Falha persistente: {str(e)[:50]}...")
            time.sleep(0.5 * (i + 1))
    
    return None

def processar_produto_rapido(produto, url_base):
    """OTIMIZADO: Processamento mais rápido de produto"""
    nome = achar_nome(produto)
    preco = achar_preco(produto)
    precopix = achar_precopix(produto)
    precodesconto = achar_precodesconto(produto)
    desconto_percentual = achar_desconto_percentual(produto)
    link_produto = achar_link(produto, url_base)

    brand, code = None, None
    detalhes_extras = {}
    
    # Apenas processar se o link for válido
    if link_produto:
        responseprodutos = baixar_url(link_produto)
        if responseprodutos:
            soup_produto = BeautifulSoup(responseprodutos.content, 'html.parser')
            brand, code = extrair_detalhes_do_json(soup_produto)
            detalhes_extras = extrair_detalhes_adicionais_da_pagina(soup_produto)

    # Usar preços da página do produto se disponíveis
    preco_final = detalhes_extras.get('unit_price_detailed') or preco
    precodesconto_final = detalhes_extras.get('discount_price_detailed') or precodesconto
    precopix_final = detalhes_extras.get('pix_price_detailed') or precopix

    unit_price = limpar_preco(preco_final)
    discount_price = limpar_preco(precodesconto_final)
    pix_price = limpar_preco(precopix_final)

    # Calcular descontos
    discount = round(unit_price - discount_price, 2) if unit_price and discount_price else None
    pix_discount = round(unit_price - pix_price, 2) if unit_price and pix_price else None

    return {
        'Nome': nome,
        'Marca': brand,
        'GTIN': code,
        'Codigo': detalhes_extras.get('product_code'),
        'Sub_categoria': detalhes_extras.get('sub_category'),
        'Preco_unitario': unit_price,
        'Preco_com_desconto': discount_price,
        'Preco_pix': pix_price,
        'Desconto': discount,
        'Desconto_com_pix': pix_discount,
        'Desconto_percentual': desconto_percentual,
        'Desconto_pix_percentual': detalhes_extras.get('pix_discount_percent'),
        'Link': link_produto
    }

def processar_pagina_completa(pagina_info):
    """NOVA FUNÇÃO: Processa uma página completa de forma otimizada"""
    pagina, total_paginas, url_base = pagina_info
    
    print(f"📄 Processando página {pagina}/{total_paginas}")
    url_pagina = f'https://www.farmaponte.com.br/saude/?p={pagina}'
    
    response = baixar_url(url_pagina)
    if not response:
        print(f"❌ Falha ao carregar página {pagina}")
        return []

    soup = BeautifulSoup(response.content, 'html.parser')
    div_produtos = soup.find_all('div', class_='li')
    print(f"🔍 Página {pagina}: {len(div_produtos)} produtos encontrados")

    # Processar produtos da página com mais workers
    produtos_pagina = []
    with ThreadPoolExecutor(max_workers=8) as executor:  # Aumentado para 8
        futures = [executor.submit(processar_produto_rapido, produto, url_base) 
                  for produto in div_produtos]
        
        for future in as_completed(futures):
            resultado = future.result()
            if resultado:
                produtos_pagina.append(resultado)

    print(f"✅ Página {pagina}: {len(produtos_pagina)} produtos processados")
    return produtos_pagina

# INÍCIO DO PROCESSAMENTO OTIMIZADO
start_time = time.time()

# --- PRIMEIRA PÁGINA para descobrir total ---
print("🔍 Descobrindo total de páginas...")
response = baixar_url(url)
soup = BeautifulSoup(response.content, 'html.parser')
total_paginas = achar_total_paginas(soup)
print(f"📊 Total de páginas encontradas: {total_paginas}")

# Preparar lista de páginas para processamento paralelo
max_paginas = total_paginas # Limite para teste inicial
paginas_info = [(p, total_paginas, url_base) for p in range(1, max_paginas + 1)]

lista_de_produtos = []

print(f"🚀 Iniciando extração paralela de {max_paginas} páginas...")
print(f"⏱️  Tempo de início: {time.strftime('%H:%M:%S')}")

# PROCESSAMENTO PARALELO DE PÁGINAS
with ThreadPoolExecutor(max_workers=4) as page_executor:  # 4 páginas em paralelo
    page_futures = [page_executor.submit(processar_pagina_completa, info) 
                   for info in paginas_info]
    
    for future in as_completed(page_futures):
        produtos_pagina = future.result()
        lista_de_produtos.extend(produtos_pagina)
        print(f"📊 Total acumulado: {len(lista_de_produtos)} produtos")

# Calcular tempo de execução
end_time = time.time()
tempo_execucao = end_time - start_time

# --- RESULTADOS FINAIS ---
print(f"\n🎉 Extração concluída em {tempo_execucao:.1f} segundos!")
print(f"📊 Coletados {len(lista_de_produtos)} produtos")
print(f"⚡ Velocidade: {len(lista_de_produtos)/tempo_execucao:.1f} produtos/segundo")

print("\n📊 Gerando tabela...")
df = pd.DataFrame(lista_de_produtos)
display(df)

print("💾 Salvando arquivo CSV...")
df.to_csv('farmaponte_otimizado.csv', index=False)
print("✅ Arquivo salvo como 'farmaponte_otimizado.csv'")

# Estatísticas de qualidade
print(f"\n📈 ESTATÍSTICAS DE QUALIDADE:")
print(f"Total de produtos: {len(df)}")
print(f"Produtos com preço unitário: {df['Preco_unitario'].notna().sum()}")
print(f"Produtos com preço desconto: {df['Preco_com_desconto'].notna().sum()}")
print(f"Produtos com preço PIX: {df['Preco_pix'].notna().sum()}")
print(f"Produtos com marca: {df['Marca'].notna().sum()}")
print(f"Produtos com GTIN: {df['GTIN'].notna().sum()}")

# Estatísticas de performance
print(f"\n⚡ ESTATÍSTICAS DE PERFORMANCE:")
print(f"Tempo total: {tempo_execucao:.1f}s")
print(f"Páginas processadas: {max_paginas}")
print(f"Produtos por página (média): {len(lista_de_produtos)/max_paginas:.1f}")
print(f"Tempo por página (média): {tempo_execucao/max_paginas:.1f}s")
print(f"Requests em cache: {len(url_cache)}")
print(f"Taxa de erro final: {consecutive_errors}")

# Exemplo de produto completo
produtos_completos = df[(df['Preco_unitario'].notna()) & 
                       (df['Marca'].notna()) & 
                       (df['GTIN'].notna())]

if len(produtos_completos) > 0:
    print(f"\n📋 Exemplo de produto com dados completos:")
    exemplo = produtos_completos.iloc[0]
    for coluna, valor in exemplo.items():
        if valor is not None and str(valor) != 'nan':
            print(f"  {coluna}: {valor}")
else:
    print(f"\n📋 Exemplo de produto coletado:")
    if len(df) > 0:
        exemplo = df.iloc[0]
        for coluna, valor in exemplo.items():
            if valor is not None and str(valor) != 'nan':
                print(f"  {coluna}: {valor}")

🔍 Descobrindo total de páginas...
📊 Total de páginas encontradas: 326
🚀 Iniciando extração paralela de 326 páginas...
⏱️  Tempo de início: 00:24:11
📄 Processando página 1/326
📄 Processando página 2/326
📄 Processando página 3/326
📄 Processando página 4/326
🔍 Página 1: 20 produtos encontrados
🔍 Página 2: 20 produtos encontrados
📊 Total de páginas encontradas: 326
🚀 Iniciando extração paralela de 326 páginas...
⏱️  Tempo de início: 00:24:11
📄 Processando página 1/326
📄 Processando página 2/326
📄 Processando página 3/326
📄 Processando página 4/326
🔍 Página 1: 20 produtos encontrados
🔍 Página 2: 20 produtos encontrados
🔍 Página 3: 20 produtos encontrados
🔍 Página 4: 20 produtos encontrados
🔍 Página 3: 20 produtos encontrados
🔍 Página 4: 20 produtos encontrados
✅ Página 1: 20 produtos processados
📄 Processando página 5/326
📊 Total acumulado: 20 produtos
✅ Página 1: 20 produtos processados
📄 Processando página 5/326
📊 Total acumulado: 20 produtos
✅ Página 2: 20 produtos processados
📄 Processa

,Nome,Marca,GTIN,Codigo,Sub_categoria,Preco_unitario,Preco_com_desconto,Preco_pix,Desconto,Desconto_com_pix,Desconto_percentual,Desconto_pix_percentual,Link
0,"Fralda Geriatrica Sensaty Premium G, 20 Unidades",Sensaty,7891522022136,Cód: 1029900,Fralda Adulta e Roupa Íntima,69.60,54.58,54.58,15.02,15.02,None,-22,https://www.farmaponte.com.br/fralda-geriatric...
1,Absorvente Geriatrico Unissex Sensaty Premium ...,Sensaty,7891522200107,Cód: 1031945,Fralda Adulta e Roupa Íntima,19.83,17.49,17.49,2.34,2.34,None,-12,https://www.farmaponte.com.br/absorvente-geria...
2,Autoteste Covid-19 Proxima Antígeno 1 Unidade,Cpmh,7898666523505,Cód: 1043408,Acessórios para a Saúde,66.56,39.10,39.10,27.46,27.46,None,-41,https://www.farmaponte.com.br/novel-coronaviru...
3,"Citrato De Sildenafila Medley 50mg, Caixa Com ...",Medley,7896422525190,Cód: 1021329,Tratamento De Disfunção Erétil,18.02,14.48,14.48,3.54,3.54,None,-20,https://www.farmaponte.com.br/citrato-de-silde...
4,Dipirona Monoidratada Solução De Uso Oral 50mg...,Biosintética - Aché,7896181911036,Cód: 800741,Analgésico Antitérmico,18.70,18.33,18.33,0.37,0.37,None,-2,https://www.farmaponte.com.br/dipirona-monoidr...
...,...,...,...,...,...,...,...,...,...,...,...,...,...
6508,"Flucomed 150mg, Caixa Com 2 Cápsulas Gelatinos...",Cimed,7896523206448,Cód: 1023550,Antimicotico,NaN,17.99,17.99,NaN,NaN,None,-2,https://www.farmaponte.com.br/flucomed-150mg-2...
6509,Amoxicilina 250mg 5ml Suspensão Oral 150ml Cim...,Cimed,7896523208473,Cód: 1018803,Antibiotico,NaN,26.49,26.49,NaN,NaN,None,-5,https://www.farmaponte.com.br/amoxicilina-250m...
6510,"Gastrol 185mg + 231,5mg + 178mg, Blíster Com 1...",Neo Química,7896622302805,Cód: 1027553,Digestivo,NaN,8.97,8.97,NaN,NaN,None,-2,https://www.farmaponte.com.br/gastrol-10past/p
6511,Nouve Collagen Ha Caixa Com 30 Sachês Com 84g ...,Cosmed,7891142207432,Cód: 1041991,Relaxante Muscular,NaN,127.30,127.30,NaN,NaN,None,-37,https://www.farmaponte.com.br/nouve-collagen-h...


💾 Salvando arquivo CSV...
✅ Arquivo salvo como 'farmaponte_otimizado.csv'

📈 ESTATÍSTICAS DE QUALIDADE:
Total de produtos: 6513
Produtos com preço unitário: 4178
Produtos com preço desconto: 6511
Produtos com preço PIX: 6507
Produtos com marca: 6455
Produtos com GTIN: 6443

⚡ ESTATÍSTICAS DE PERFORMANCE:
Tempo total: 766.7s
Páginas processadas: 326
Produtos por página (média): 20.0
Tempo por página (média): 2.4s
Requests em cache: 6840
Taxa de erro final: 0

📋 Exemplo de produto com dados completos:
  Nome: Fralda Geriatrica Sensaty Premium G, 20 Unidades
  Marca: Sensaty
  GTIN: 7891522022136
  Codigo: Cód: 1029900
  Sub_categoria: Fralda Adulta e Roupa Íntima
  Preco_unitario: 69.6
  Preco_com_desconto: 54.58
  Preco_pix: 54.58
  Desconto: 15.02
  Desconto_com_pix: 15.02
  Desconto_pix_percentual: -22
  Link: https://www.farmaponte.com.br/fralda-geriatrica-sensaty-premium-g-20-unidades/p


In [ ]:
# 🎛️ ADAPTIVE CONCURRENCY SYSTEM
# This system automatically adjusts concurrent operations based on performance

import psutil
import time
from threading import Lock

class AdaptiveConcurrencyManager:
    def __init__(self):
        self.current_page_workers = 2      # Start conservative
        self.current_product_workers = 4
        self.max_page_workers = 8
        self.max_product_workers = 16
        self.min_workers = 1
        
        self.success_count = 0
        self.error_count = 0
        self.response_times = []
        self.adjustment_lock = Lock()
        
        # Performance thresholds
        self.max_cpu_percent = 85
        self.max_memory_percent = 80
        self.max_error_rate = 0.1  # 10% error rate
        self.target_response_time = 3.0  # seconds
        
    def get_current_settings(self):
        """Get current concurrency settings"""
        return {
            'page_workers': self.current_page_workers,
            'product_workers': self.current_product_workers,
            'total_concurrent': self.current_page_workers * self.current_product_workers
        }
    
    def record_success(self, response_time):
        """Record a successful operation"""
        with self.adjustment_lock:
            self.success_count += 1
            self.response_times.append(response_time)
            
            # Keep only recent response times (last 50)
            if len(self.response_times) > 50:
                self.response_times = self.response_times[-50:]
    
    def record_error(self):
        """Record a failed operation"""
        with self.adjustment_lock:
            self.error_count += 1
    
    def should_increase_concurrency(self):
        """Check if we can safely increase concurrency"""
        if self.success_count + self.error_count < 20:
            return False  # Not enough data yet
            
        # Check system resources
        cpu_usage = psutil.cpu_percent(interval=0.1)
        memory_usage = psutil.virtual_memory().percent
        
        if cpu_usage > self.max_cpu_percent or memory_usage > self.max_memory_percent:
            return False
            
        # Check error rate
        total_operations = self.success_count + self.error_count
        error_rate = self.error_count / total_operations
        
        if error_rate > self.max_error_rate:
            return False
            
        # Check response times
        if self.response_times:
            avg_response_time = sum(self.response_times) / len(self.response_times)
            if avg_response_time > self.target_response_time:
                return False
                
        return True
    
    def should_decrease_concurrency(self):
        """Check if we need to decrease concurrency"""
        if self.success_count + self.error_count < 10:
            return False
            
        # Check system resources
        cpu_usage = psutil.cpu_percent(interval=0.1)
        memory_usage = psutil.virtual_memory().percent
        
        if cpu_usage > self.max_cpu_percent or memory_usage > self.max_memory_percent:
            return True
            
        # Check error rate
        total_operations = self.success_count + self.error_count
        error_rate = self.error_count / total_operations
        
        if error_rate > self.max_error_rate:
            return True
            
        # Check response times
        if len(self.response_times) >= 10:
            avg_response_time = sum(self.response_times[-10:]) / 10
            if avg_response_time > self.target_response_time * 1.5:
                return True
                
        return False
    
    def adjust_concurrency(self):
        """Automatically adjust concurrency based on performance"""
        with self.adjustment_lock:
            if self.should_decrease_concurrency():
                # Decrease concurrency
                self.current_product_workers = max(self.min_workers, self.current_product_workers - 1)
                if self.current_product_workers == self.min_workers:
                    self.current_page_workers = max(self.min_workers, self.current_page_workers - 1)
                
                print(f"🔻 Reducing concurrency: {self.current_page_workers} pages × {self.current_product_workers} products")
                
            elif self.should_increase_concurrency():
                # Increase concurrency
                if self.current_product_workers < self.max_product_workers:
                    self.current_product_workers += 1
                elif self.current_page_workers < self.max_page_workers:
                    self.current_page_workers += 1
                    
                print(f"🔺 Increasing concurrency: {self.current_page_workers} pages × {self.current_product_workers} products")
            
            # Reset counters for next evaluation
            self.success_count = 0
            self.error_count = 0
    
    def get_performance_report(self):
        """Get current performance metrics"""
        total_ops = self.success_count + self.error_count
        error_rate = (self.error_count / total_ops * 100) if total_ops > 0 else 0
        avg_response = (sum(self.response_times) / len(self.response_times)) if self.response_times else 0
        
        return {
            'total_operations': total_ops,
            'error_rate_percent': error_rate,
            'avg_response_time': avg_response,
            'cpu_usage': psutil.cpu_percent(interval=0.1),
            'memory_usage': psutil.virtual_memory().percent,
            'current_settings': self.get_current_settings()
        }

# Example usage:
print("🎛️ Adaptive Concurrency Manager Example")
print("This system automatically adjusts workers based on:")
print("  • CPU and memory usage")
print("  • Error rates") 
print("  • Response times")
print("  • Server performance")
print()
print("Recommended approach:")
print("1. Start with conservative settings (2 pages × 4 products)")
print("2. Let the system monitor performance")
print("3. Automatically scale up when safe")
print("4. Scale down when hitting limits")
print()
print("Manual override guidelines:")
print("• 1-2 pages × 2-4 products = 2-8 total (Conservative)")
print("• 3-4 pages × 5-8 products = 15-32 total (Moderate)")  
print("• 5-8 pages × 8-12 products = 40-96 total (Aggressive)")
print()
print("⚠️ Never exceed your system's capabilities!")
print("Monitor CPU, memory, and error rates closely.")